In [11]:
# Импорт библиотек
import numpy as np
import matplotlib.pyplot as plt
from sklearn.tree import DecisionTreeRegressor,DecisionTreeClassifier, plot_tree
from ipywidgets import interact, widgets
from sklearn.datasets import make_regression,make_classification

**Дерево решений** - это алгоритм машинного обучения, который:
- Строит древовидную модель решений
- Используется для задач классификации и регрессии
- Работает по принципу "разделяй и властвуй"

Для датасета с:

Признаками: $X = {x_1, ..., x_n}$

Целевой переменной: $y$ (классы или числовые значения)

$m$ образцов: $(X_i, y_i), i=1..m$

#### Для классификации:
**1. Индекс Джини (Gini Impurity)**

$$
G = \sum_{i=1}^{C} p_i(1-p_i) = 1 - \sum_{i=1}^{C} p_i^2
$$
где:

$C$ — количество классов

$p_i$ — доля образцов класса $i$ в узле


**2. Энтропия и Information Gain**

$$
H = -\sum_{i=1}^{C} p_i \log_2 p_i
$$

$$
IG = H_{\text{parent}} - \left( \frac{N_{\text{left}}}{N}H_{\text{left}} + \frac{N_{\text{right}}}{N}H_{\text{right}} \right)
$$


In [14]:
X, y = make_classification()

# Создаем и обучаем модель для классификации
tree = DecisionTreeClassifier()
tree.fit(X, y)

DecisionTreeClassifier()

###Для регрессии:

$$
MSE = \frac{1}{N}\sum_{i=1}^{N}(y_i - \hat{y_i})^2
$$

In [15]:
# Генерация данных
X, y = make_regression(n_features=1, n_samples=100, noise=20, random_state=42)
X = X * 50 + 50  # Масштабируем к диапазону площадей [0-100 м²]
y = y * 20000 + 100000  # Масштабируем к ценам [50k-150k руб]

# Создаем и обучаем модель для классификации
tree = DecisionTreeRegressor()
tree.fit(X, y)

DecisionTreeRegressor()


##  Основные параметры

###  Критерий разделения
- **Для классификации**:
  - `gini` (по умолчанию) - быстрый расчет нечистоты
  - `entropy` - более чувствителен к распределению классов
- **Для регрессии**:
  - `squared_error` - минимизирует среднеквадратичную ошибку
  - `absolute_error` - устойчив к выбросам

###  Контроль  дерева
| Параметр | Описание | Рекомендуемые значения |
|----------|----------|------------------------|
| `max_depth` | Максимальная глубина | 3-7 (баланс точности и интерпретируемости) |
| `min_samples_split` | Минимум образцов для разделения узла | 2-20 |
| `min_samples_leaf` | Минимум образцов в конечном узле | 1-10 |
| `class_weight='balanced'` |   автоматическая балансировка| |



#Перед вами интерактивная песочница для экспериментов с деревом решений. Меняйте параметры и сразу видите результат!

##  Что можно настроить:
1. **Глубина дерева** - от 1 до 10 уровней  
   *Чем больше - тем сложнее модель*

2. **Метрика качества**


3. **Минимум в листе** - от 1 до 20 примеров  
   *Чем больше - тем проще модель*

In [18]:


# Функция для интерактивного отображения
def interactive_tree(max_depth=2, min_samples_leaf=2, criterion='squared_error'):
    tree = DecisionTreeRegressor(
        max_depth=max_depth,
        min_samples_leaf=min_samples_leaf,
        criterion=criterion,
        random_state=42
    )
    tree.fit(X, y)

    fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(16, 6))

    x_test = np.linspace(X.min(), X.max(), 300).reshape(-1, 1)
    y_pred = tree.predict(x_test)

    ax1.scatter(X, y, alpha=0.7, label='Данные')
    ax1.plot(x_test, y_pred, 'r-', linewidth=2, label='Предсказание')
    ax1.set_xlabel('Площадь (м²)')
    ax1.set_ylabel('Цена (руб)')
    ax1.set_title('Предсказания дерева')
    ax1.legend()
    ax1.grid(True)

    plot_tree(tree, filled=True, feature_names=['Площадь'], ax=ax2)
    ax2.set_title('Структура дерева')

    plt.tight_layout()
    plt.show()

    print(f"Количество листьев: {tree.get_n_leaves()}")
    print(f"Глубина дерева: {tree.get_depth()}")
    if tree.tree_.node_count > 0:
        print(f"Пример правила: Если площадь <= {tree.tree_.threshold[0]:.1f} м²")

interact(interactive_tree,
         max_depth=widgets.IntSlider(value=2, min=1, max=10, description='Глубина:'),
         min_samples_leaf=widgets.IntSlider(value=2, min=1, max=20, description='Минимум в листе:'),
         criterion=widgets.Dropdown(
             options=['squared_error', 'friedman_mse', 'absolute_error'],
             value='squared_error',
             description='Критерий:'
         ));

interactive(children=(IntSlider(value=2, description='Глубина:', max=10, min=1), IntSlider(value=2, descriptio…